In [56]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import re
import plotly as pl 
import matplotlib.pyplot as plt
from pathlib import Path

# Import db related librarys
import psycopg2
import sys
import boto3
import os
import sqlalchemy
import json
import csv
from getpass import getpass

In [74]:
# Loading country_lat_long data from S3 Bucket.
lat_long_data = "https://chocolate-final-project-21.s3.us-east-2.amazonaws.com/country_and_usa_states_lat_long.csv"
lat_long_df = pd.read_csv(lat_long_data)

#file_path = Path('country_and_usa_states_lat_long.csv')
#df = pd.read_csv(file_path)

#lat_long_csv = lat_long_df.to_csv()

lat_long_df.head()

,country_code,latitude,longitude,country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [75]:
# Load flavors_of_cacoa version s3
clean_flavors_data = "https://chocolate-final-project-21.s3.us-east-2.amazonaws.com/clean_flavors_of_cacao.csv"
clean_flavors_df = pd.read_csv(clean_flavors_data)

clean_flavors_df.head()

,Company,Bean_Origin_or_Bar_Name,REF,Review_Date,Cocoa_Percent,Company_Location,Rating,Bean_Type,Broad_Bean_Origin,Ingredients,Most_Memorable_Characteristics,country_code,latitude,longitude,continent
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe,"4- B,S,C,L","sweet, chocolatey, vegetal",ST,0.186360,6.613081,Africa
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo,"4- B,S,C,L","burnt wood, earthy, choco",TG,8.619543,0.824782,Africa
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo,"4- B,S,C,L","roasty, acidic, nutty",TG,8.619543,0.824782,Africa
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo,"4- B,S,C,L","mild profile, chocolaty, spice",TG,8.619543,0.824782,Africa
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru,"4- B,S,C,L","grainy texture, cocoa, sweet",PE,-9.189967,-75.015152,South America


In [76]:
# Configure connection parameters to AWS RDS
def connect_to_db():
    
    connection = None
    #enter_password = getpass('Enter database password')
    
    try:
        print('Connecting to the PostgreSQL database...')
        connection = psycopg2.connect(
            host = "database-chocolate.cafzzay3t2tr.us-east-2.rds.amazonaws.com",
            port = 5432,
            user = 'postgres',
            password = getpass('Enter database password'),
            database = 'postgres'
            )
        print('Connection successful')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    return connection



In [77]:
# Call the connect_to_db to connect to database
connection = connect_to_db()


Connecting to the PostgreSQL database...
Enter database password········
Connection successful


In [78]:
# Create cursor to perform database operations
cursor = connection.cursor()

# If curser object = 0, then connection was successfully established
cursor

<cursor object at 0x7fdfbfa83850; closed: 0>

In [74]:
# Create location table - this table has been set up (already exists)

#cursor.execute(""" CREATE TABLE IF NOT EXISTS location_table(
#country_code text,
#latitude float,
#longitude float,
#country text)""")

#connection.commit()

In [123]:
# Create clean_flavors_of_cacao table on postgres.

#country text)""")
#cursor.execute(""" CREATE TABLE IF NOT EXISTS clean_flavors_table(
#Company text NOT NULL,
#Bean_Origin_or_Bar_Name text NULL,
#REF text NOT NULL,
#Review_Date text NOT NULL,
#Cocoa_Percent float NOT NULL,
#Company_Location text NULL,
#Rating float NOT NULL,
#Bean_Type text NULL,
#Broad_Bean_Origin text NULL,
#Ingredients text NULL,
#Most_Memorable_Characteristics text NULL ,
#country_code text NULL,
#latitude float NULL,
#longitude float NULL,
#continent text NULL,
#PRIMARY KEY 
#)""")

#connection.commit()
#print("Table created successfully........")


In [124]:
#sql = """
#DROP TABLE clean_flavors_table
#"""

#cursor.execute(sql)

In [122]:
#sql = """
#DROP TABLE location_table
#"""

#cursor.execute(sql)

In [17]:
# Load data into postgres table from csv currently - have not figured out how to directly open from s3.
# No need to re-run this cell as csv file has already been imported into table.

#with open ('country_and_usa_states_lat_long.csv', 'r') as row:
#           next(row) # skip the header row
#           cursor.copy_from(row, 'location_table', sep = ',')
                   
#connection.commit()

In [10]:
# Load clean_flavors data into postgres table from csv currently - have not figured out how to directly open from s3.

#with open ('clean_flavors_of_cacao.csv', 'r',encoding='utf-8') as row:
#           next(row) # skip the header row
#           cursor.copy_from(row, 'clean_flavors_table', sep = ',')
                   
#connection.commit()

In [62]:
# Query the location_table from AWS database and store in dataframe - verify there is data in the table.
sql = """
SELECT * FROM location_table
"""

location_table_from_db = pd.read_sql(sql, con=connection)

In [63]:
# Display the location_table queried from database
location_table_from_db.head(5)

,country_code,latitude,longitude,broad_bean_origin_country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [127]:
# To end transaction
# connection.rollback()

In [64]:
# Query the clean_flavours_table from AWS database and store in dataframe.
sql = """
SELECT * FROM clean_flavors_table
"""

clean_flavors_from_db = pd.read_sql(sql, con=connection)

In [65]:
clean_flavors_from_db.head()

,company,bean_origin_or_bar_name,REF,review_date,cocoa_percent,company_location,rating,bean_type,broad_bean_origin_country,ingredients,most_memorable_characteristics,continent
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe,"4- B,S,C,L","sweet, chocolatey, vegetal",Africa
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo,"4- B,S,C,L","burnt wood, earthy, choco",Africa
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo,"4- B,S,C,L","roasty, acidic, nutty",Africa
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo,"4- B,S,C,L","mild profile, chocolaty, spice",Africa
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru,"4- B,S,C,L","grainy texture, cocoa, sweet",South America


In [66]:
# Make sure connection is closed. If object returns 1, then connection is closed.

cursor

<cursor object at 0x7fdfbfa83550; closed: 0>

In [80]:
# Join the 2 tables together on broad_bean_origin_country

sql = """ SELECT clean_flavors_table .*, location_table.country_code,location_table.longitude, location_table.latitude
FROM clean_flavors_table
INNER JOIN location_table
ON clean_flavors_table.broad_bean_origin_country = location_table.broad_bean_origin_country;
"""

# Store the joined tables in dataframe
joined_tables = pd.read_sql(sql, con=connection)

# View the new dataframe combined from two sql tables
joined_tables.head(20)

,company,bean_origin_or_bar_name,REF,review_date,cocoa_percent,company_location,rating,bean_type,broad_bean_origin_country,ingredients,most_memorable_characteristics,continent,country_code,longitude,latitude
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe,"4- B,S,C,L","sweet, chocolatey, vegetal",Africa,ST,6.613081,0.186360
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo,"4- B,S,C,L","burnt wood, earthy, choco",Africa,TG,0.824782,8.619543
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo,"4- B,S,C,L","roasty, acidic, nutty",Africa,TG,0.824782,8.619543
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo,"4- B,S,C,L","mild profile, chocolaty, spice",Africa,TG,0.824782,8.619543
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru,"4- B,S,C,L","grainy texture, cocoa, sweet",South America,PE,-75.015152,-9.189967
5,A. Morin,Carenero,1315,2014,70.0,France,2.75,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
6,A. Morin,Cuba,1315,2014,70.0,France,3.50,missing,Cuba,"4- B,S,C,L","sliglty dry, papaya",Caribbean,CU,-77.781167,21.521757
7,A. Morin,Sur del Lago,1315,2014,70.0,France,3.50,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
8,A. Morin,Puerto Cabello,1319,2014,70.0,France,3.75,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
9,A. Morin,Pablino,1319,2014,70.0,France,4.00,missing,Peru,"4- B,S,C,L","delicate, hazelnut, brownie",South America,PE,-75.015152,-9.189967


In [81]:
# Close connection
cursor.close()

In [82]:
# Check that connection is closed where object returned = -1
cursor

<cursor object at 0x7fdfbfa83850; closed: -1>